In [1]:
!pip install -q gdown

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import albumentations as A
import seaborn as sns
import sklearn
import datetime
import json
import xml.etree.ElementTree as ET
import io
import os
import cv2 as cv
from random import sample
import shutil
import gdown
from PIL import Image
from sklearn.metrics import confusion_matrix, roc_curve
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, Activation, Add, Permute, LeakyReLU, Reshape
from tensorflow.keras.layers import Input, Conv2D, Dense, BatchNormalization, MaxPooling2D, InputLayer, Flatten, Dropout, RandomRotation, RandomContrast, RandomFlip, Resizing, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError, BinaryAccuracy, Precision, Recall, AUC, FalsePositives, TruePositives, FalseNegatives, TrueNegatives, CategoricalAccuracy, TopKCategoricalAccuracy
from tensorflow.keras.losses import MeanSquaredError, Huber, MeanAbsoluteError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, CSVLogger, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.regularizers import L2, L1
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Example, Features, Feature

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
strategy = tf.distribute.MirroredStrategy()
print('DEVICES AVAILABLE: {}'.format(strategy.num_replicas_in_sync))

DEVICES AVAILABLE: 1


# Dataset Download

In [4]:
gdown.download(id='1D4Tvi7EIEy1Ggiu52rN902pq2QnhP4hw', output='/kaggle/working/')

Downloading...
From (uriginal): https://drive.google.com/uc?id=1D4Tvi7EIEy1Ggiu52rN902pq2QnhP4hw
From (redirected): https://drive.google.com/uc?id=1D4Tvi7EIEy1Ggiu52rN902pq2QnhP4hw&confirm=t&uuid=31de75e5-f7cb-4010-a3c8-95adff8256ab
To: /kaggle/working/Wastebasket_trash_detecation.zip
100%|██████████| 441M/441M [00:04<00:00, 91.2MB/s] 


'/kaggle/working/Wastebasket_trash_detecation.zip'

In [5]:
!unzip -q /kaggle/working/Wastebasket_trash_detecation.zip -d /kaggle/working/

In [6]:
!rm -rf /kaggle/working/Wastebasket_trash_detecation.zip

# Dataset Preparation

In [7]:
class_names = ['Bottle', 'Box', 'CAN', 'Can', 'Dustbin', 'LDPE', 'Tempat-sampah', 'Trash-Can', 'bin', 'bottle', 'bottled-drinks', 'can', 'canned-drinks', 'fake bag', 'glass', 'green bag', 'litter-bins', 'people', 'pet', 'plastic-bottles', 'plastic_cup', 'plasticbottle', 'tetrapack', 'trash', 'trashcan', 'wastebin', 'white bag']

In [8]:
B=2
N_CLASSES=27
H,W =640,640
SPLIT_SIZE=int(H/32)
BATCH_SIZE=32

In [9]:
train_im_path = []
train_annot_path = []
val_im_path = []
val_annot_path = []

for name in os.listdir('/kaggle/working/Wastebasket_trash_detecation/images/train/'):
    train_im_path.append(name)
    train_annot_path.append(name[:-3] + 'txt')
    
for name in os.listdir('/kaggle/working/Wastebasket_trash_detecation/images/val/'):
    val_im_path.append(name)
    val_annot_path.append(name[:-3] + 'txt')
    
print(len(train_im_path)) 
print(len(train_annot_path))
print(len(val_im_path)) 
print(len(val_annot_path))

6002
6002
1504
1504


In [10]:
train_ds1 = tf.data.Dataset.from_tensor_slices((train_im_path, train_annot_path))
val_ds1 = tf.data.Dataset.from_tensor_slices((val_im_path, val_annot_path))

In [11]:
for i, j in train_ds1.take(1):
    print(i)
    print(j)

tf.Tensor(b'singapore-december-8th-2019-dustbin-260nw-1582927012_jpg.rf.22e99ed2e2be257d06e56e6e9a376623.jpg', shape=(), dtype=string)
tf.Tensor(b'singapore-december-8th-2019-dustbin-260nw-1582927012_jpg.rf.22e99ed2e2be257d06e56e6e9a376623.txt', shape=(), dtype=string)


In [12]:
def get_train_bboxes(filepath):
    if type(filepath) == bytes:
        filepath = filepath.decode('utf-8')
    bounding_boxes = []
    with open('/kaggle/working/Wastebasket_trash_detecation/labels/train/'+ filepath, 'r') as f:
        lines = f.readlines()
        
    for i in range(len(lines)):
        annots = lines[i].split()
        new_annot = [float(annots[1]) ,float(annots[2]), float(annots[3]), float(annots[4]), float(annots[0])]
        
        bounding_boxes.append(new_annot)
        
    return tf.convert_to_tensor(bounding_boxes, dtype=tf.float32)

In [13]:
get_train_bboxes('00fd4f937015974aaf935a637211ba8e_jpeg.rf.9c5c07e0572a9b346f88d840e52f7f8d.txt')

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.5054687 ,  0.4015625 ,  0.41640624,  0.6609375 , 11.        ],
       [ 0.7390625 ,  0.6328125 ,  0.41953126,  0.6578125 , 11.        ],
       [ 0.28984374,  0.6429688 ,  0.41640624,  0.6546875 , 11.        ]],
      dtype=float32)>

In [14]:
def get_val_bboxes(filepath):
    if type(filepath) == bytes:
        filepath = filepath.decode('utf-8')
    bounding_boxes = []
    with open('/kaggle/working/Wastebasket_trash_detecation/labels/val/'+ filepath, 'r') as f:
        lines = f.readlines()
        
    for i in range(len(lines)):
        annots = lines[i].split()
        new_annot = [float(annots[1]) ,float(annots[2]), float(annots[3]), float(annots[4]), float(annots[0])]
        
        bounding_boxes.append(new_annot)
        
    return tf.convert_to_tensor(bounding_boxes, dtype=tf.float32)

In [15]:
def get_train_img_and_bboxes(img_path, annot_path):
    img = tf.io.decode_image(tf.io.read_file('/kaggle/working/Wastebasket_trash_detecation/images/train/'+ img_path), expand_animations=False)
    
    bboxes = tf.numpy_function(func=get_train_bboxes, inp=[annot_path], Tout=tf.float32)
    
    return img, bboxes


In [16]:
def get_val_img_and_bboxes(img_path, annot_path):
    img = tf.io.decode_image(tf.io.read_file('/kaggle/working/Wastebasket_trash_detecation/images/val/'+ img_path), expand_animations=False)
    
    bboxes = tf.numpy_function(func=get_val_bboxes, inp=[annot_path], Tout=tf.float32)
    
    return img, bboxes

In [17]:
train_ds2 = train_ds1.map(get_train_img_and_bboxes)
val_ds2 = val_ds1.map(get_val_img_and_bboxes)

In [18]:
for i, j in train_ds2.take(1):
    print(i.shape)
    print(j)

(640, 640, 3)
tf.Tensor(
[[0.62109375 0.14765625 0.2859375  0.2953125  4.        ]
 [0.87890625 0.728125   0.23828125 0.45703125 4.        ]], shape=(2, 5), dtype=float32)


# Final Dataset Preparation

In [19]:
def output_bboxes(bboxes):
    output_label = np.zeros((SPLIT_SIZE, SPLIT_SIZE, N_CLASSES+5))
    
    for b in range(len(bboxes)):
        grid_x = bboxes[...,b,0]*SPLIT_SIZE
        grid_y = bboxes[...,b,1]*SPLIT_SIZE
        
        i = min(int(grid_x), SPLIT_SIZE-1)
        j = min(int(grid_y), SPLIT_SIZE-1)
        
        output_label[i, j, 0:5] = [1., grid_x%1, grid_y%1, bboxes[...,b,2], bboxes[...,b,3]]
        output_label[i, j, 5+int(bboxes[...,b,4])] = 1.
        
    return tf.convert_to_tensor(output_label, tf.float32)

In [20]:
def process_data(img, boxes):
    img = tf.image.random_brightness(img, max_delta=50)
    img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
    img = tf.cast(tf.image.resize(img, [H,W]), dtype=tf.float32)
    img = tf.clip_by_value(img, 0, 255)
    
    labels = tf.numpy_function(func=output_bboxes, inp=[boxes], Tout=tf.float32)
    
    return img, labels

In [21]:
def process_data_val(img, boxes):
    img = tf.cast(tf.image.resize(img, [H,W]), dtype=tf.float32)
    
    labels = tf.numpy_function(func=output_bboxes, inp=[boxes], Tout=tf.float32)
    
    return img, labels

In [22]:
train_ds3 = train_ds2.map(process_data)
val_ds3 = val_ds2.map(process_data_val)

In [23]:
for i,j in train_ds3.take(1):
    print(i.shape)
    print(j.shape)

(640, 640, 3)
(20, 20, 32)


In [24]:
train_ds = train_ds3.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds3.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [25]:
for i,j in train_ds.take(1):
    print(i.shape)
    print(j.shape)

(32, 640, 640, 3)
(32, 20, 20, 32)


# Model

In [26]:
OUTPUT_DIMS = B*5 + N_CLASSES
NUM_FILTERS = 512
N_EPOCHS=135

In [27]:
base_model = tf.keras.applications.efficientnet.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_shape=(H,W,3)
)

27018416/27018416 [==============================] - 2s 0us/step


In [28]:
base_model.trainable = False

In [29]:
model = tf.keras.Sequential([
        base_model,

        Conv2D(NUM_FILTERS, (3,3), padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),

        Conv2D(NUM_FILTERS, (3,3), padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),

        Conv2D(NUM_FILTERS, (3,3), padding='same', kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),

        Conv2D(NUM_FILTERS, (3,3), padding='same', kernel_initializer='he_normal'),
        LeakyReLU(alpha=0.1),

        Flatten(),

        Dense(NUM_FILTERS, kernel_initializer='he_normal'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),

        Dropout(0.5),

        Dense(SPLIT_SIZE*SPLIT_SIZE*OUTPUT_DIMS, activation='sigmoid'),

        Reshape((SPLIT_SIZE, SPLIT_SIZE, OUTPUT_DIMS))
    ])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional  (None, 20, 20, 1280)      6575239   
 )                                                               
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 512)       5898752   
                                                                 
 batch_normalization (Batch  (None, 20, 20, 512)       2048      
 Normalization)                                                  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 20, 20, 512)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 512)       2359808   
                                                                 
 batch_normalization_1 (Bat  (None, 20, 20, 512)       2

# Loss Function

In [30]:
def compute_iou(box_1, box_2):
    box_1_c = tf.stack([
      box_1[...,0] - box_1[...,2]/2.0,
      box_1[...,1] - box_1[...,3]/2.0,
      box_1[...,0] + box_1[...,2]/2.0,
      box_1[...,1] + box_1[...,3]/2.0,
    ], axis=-1)

    box_2_c = tf.stack([
        box_2[...,0] - box_2[...,2]/2.0,
        box_2[...,1] - box_2[...,3]/2.0,
        box_2[...,0] + box_2[...,2]/2.0,
        box_2[...,1] + box_2[...,3]/2.0,
    ], axis=-1)

    luc = tf.maximum(box_1_c[...,:2], box_2_c[...,:2])
    rlc = tf.minimum(box_1_c[...,2:], box_2_c[...,2:])

    intersection = tf.maximum(0.0, rlc-luc)
    intersection_area = intersection[...,0] * intersection[...,1]

    square_1 = box_1[...,2]*box_1[...,3]
    square_2 = box_2[...,2]*box_2[...,3]

    union_area = tf.maximum(square_1 + square_2 - intersection_area, 1e-10)

    return tf.clip_by_value(intersection_area/union_area, 0.0, 1.0)

In [31]:
def difference(x,y):
    return tf.reduce_sum(tf.square(y-x))

In [33]:
def yolo_loss(y_true, y_pred):
    target = y_true[...,0]
    ## Object loss
    y_pred_extract = tf.gather_nd(y_pred, tf.where(target[:]==1))
    y_target_extract = tf.gather_nd(y_true, tf.where(target[:]==1))

    rescaler = tf.where(target[:]==1)*SPLIT_SIZE
    upscaler_1 = tf.concat([rescaler[:,1:], tf.zeros([len(rescaler), 2], dtype=tf.int64)], axis=-1)

    target_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                           repeats=[len(rescaler)], axis=0)*tf.cast(y_target_extract[...,1:5], dtype = tf.float32)
    pred_1_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                          repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,1:5], dtype = tf.float32)
    pred_2_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                          repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,6:10], dtype = tf.float32)

    target_orig = tf.cast(upscaler_1, dtype = tf.float32)+target_upscaler_2
    pred_1_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_1_upscaler_2
    pred_2_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_2_upscaler_2

    mask =tf.cast(tf.math.greater(compute_iou(target_orig,pred_2_orig),
                                             compute_iou(target_orig,pred_1_orig)),dtype=tf.int32)

    y_pred_joined=tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[...,0],axis=0),
                            tf.expand_dims(y_pred_extract[...,5],axis=0)],axis=0))

    obj_pred = tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))

    object_loss = difference(tf.cast(obj_pred,dtype =tf.float32)
                                ,tf.cast(tf.ones([len(rescaler)]),dtype=tf.float32))

    ####################### For No object
    y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==0))
    y_target_extract = tf.zeros(len(y_pred_extract))

    no_object_loss_1 = difference(tf.cast(y_pred_extract[...,0],dtype =tf.float32)
                                ,tf.cast(y_target_extract,dtype=tf.float32))

    no_object_loss_2 = difference(tf.cast(y_pred_extract[...,5],dtype =tf.float32)
                                ,tf.cast(y_target_extract,dtype=tf.float32))

    no_object_loss = no_object_loss_1+no_object_loss_2

    ######################## For OBject class loss
    y_pred_extract = tf.gather_nd(y_pred[...,10:],tf.where(target[:]==1))
    class_extract = tf.gather_nd(y_true[...,5:],tf.where(target[:]==1))

    class_loss = difference(tf.cast(y_pred_extract,dtype =tf.float32)
                                    ,tf.cast(class_extract,dtype=tf.float32))

    ######################### For object bounding box loss
    y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==1))
    centre_joined=tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=1)
    centre_pred = tf.gather_nd(centre_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
    centre_target = tf.gather_nd(y_true[...,1:3], tf.where(target[:]==1))

    centre_loss = difference(centre_pred,centre_target)

    size_joined=tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=1)

    size_pred = tf.gather_nd(size_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
    size_target = tf.gather_nd(y_true[...,3:5], tf.where(target[:]==1))

    size_loss = difference(tf.math.sqrt(tf.math.abs(size_pred)),tf.math.sqrt(tf.math.abs(size_target)))
    box_loss = centre_loss+size_loss

    lambda_coord = 5.0
    lambda_no_obj = 0.5

    loss = object_loss + (lambda_no_obj*no_object_loss)+ tf.cast(lambda_coord*box_loss,dtype=tf.float32)+ tf.cast(class_loss,dtype=tf.float32)
    return loss

# Callbacks

In [34]:
!mkdir /kaggle/working/model

In [35]:
model_checkpoint_path = './model/yolo_v1.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = model_checkpoint_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [36]:
def scheduler(epoch, lr):
    if epoch < 40:
        return 1e-3
    elif epoch>=40 and epoch<80:
        return 5e-4
    else:
        return 1e-4

In [38]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Training

In [39]:
model.compile(loss = yolo_loss, optimizer=Adam(1e-3))

In [ ]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    verbose = 1,
    epochs = N_EPOCHS,
    callbacks = [callback, lr_callback]
)

# Testing

In [43]:
model.load_weights(model_checkpoint_path)

In [49]:
!zip -r model.zip /kaggle/working/model

  adding: kaggle/working/model/ (stored 0%)
  adding: kaggle/working/model/yolo_v1.h5 (deflated 8%)


In [50]:
from IPython.display import FileLink
FileLink(r'model.zip')

/kaggle/working/model.zip